In [2]:
from pathlib import Path
from tempfile import TemporaryDirectory

import torch
from loguru import logger
from mido import MidiFile

from yoshimidi import player
from yoshimidi.data.parse import midi_parsing, one_hot_parsing, token_parsing, track_parsing

pygame 2.5.0 (SDL 2.28.0, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
PATH_INDEX = 1
TRACK_INDEX = 11
# PATH_INDEX = 2
# TRACK_INDEX = 0
CHANNEL_INDEX = None
midi_path = list(Path("../out/dataset/01_raw/lmd_full/").rglob("*.mid"))[PATH_INDEX]

In [3]:
player.play(midi_path)

2023-08-03 13:19:57.509 | INFO     | yoshimidi.player:play:12 - Playing ../out/dataset/01_raw/lmd_full/9/9daedd9c5a390069a9cdf98bc1e05565.mid
2023-08-03 13:20:08.577 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [21]:
midi_file = MidiFile(midi_path)
logger.info(f"num tracks: {len(midi_file.tracks)}")
logger.info(f"track num messages: {[len(track) for track in midi_file.tracks]}")
midi_track = midi_file.tracks[TRACK_INDEX]
tempo = track_parsing.parse_tempo(midi_file)
ym_track = track_parsing.from_midi(midi_track, ticks_per_beat=midi_file.ticks_per_beat, tempo=tempo)
assert ym_track is not None
ym_tokens = [token_parsing.from_channel(channel) for channel in ym_track.channels.values()]
if CHANNEL_INDEX is not None:
    ym_tokens = [ym_tokens[CHANNEL_INDEX]]
ym_one_hot = one_hot_parsing.from_tokens(ym_tokens[0], device=torch.device("cpu"), dtype=torch.float32)

ym_track_recons = track_parsing.from_tokens(ym_tokens)
midi_file_recons = midi_parsing.from_tracks([ym_track_recons])

with TemporaryDirectory() as temp:
    midi_path_recons = Path(temp) / "recons.mid"
    midi_file_recons.save(midi_path_recons)
    player.play(midi_path_recons)

2023-08-03 13:50:26.628 | INFO     | __main__:<module>:2 - num tracks: 13
2023-08-03 13:50:26.629 | INFO     | __main__:<module>:3 - track num messages: [2, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 311, 47]
2023-08-03 13:50:26.634 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:131 - on
2023-08-03 13:50:26.635 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:131 - pause
2023-08-03 13:50:26.635 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:161 - adding secs
2023-08-03 13:50:26.635 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:163 - [  2   0   0 120]
2023-08-03 13:50:26.635 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:131 - off
2023-08-03 13:50:26.636 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:135 - added buffer with 0.25890903254474606
2023-08-03 13:50:26.636 | INFO     | yoshimidi.data.parse.track_parsing:from_tokens:131 - pause
2023-08-03 13:50:26.636 | INFO     | yoshimidi.data.parse.track_parsing:from_

528 264


2023-08-03 13:50:32.203 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [16]:
ym_track_recons

Track(channels={0: Channel(notes=[Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0), Note(note=0, kind='on', velocity=127, time_delta_secs=0.0), Note(note=0, kind='off', velocity=127, time_delta_secs=0.0)